In [56]:
from os.path import join as opj
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.interfaces.spm import (OneSampleTTestDesign, EstimateModel,
                                   EstimateContrast, Threshold)
from nipype.interfaces.utility import IdentityInterface
from nipype.pipeline.engine import Workflow, Node

from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import MapNode

# SPM standalone
matlab_cmd = '/usr/local/spm12/run_spm12.sh /usr/local/MATLAB/MATLAB_Compiler_Runtime/v713 script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

# Specification to MATLAB
#from nipype.interfaces.matlab import MatlabCommand
#MatlabCommand.set_default_paths('/usr/local/MATLAB/R2014b/toolbox/spm12')
#MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")


NameError: name 'spm' is not defined

In [49]:
experiment_dir = '/media/lmn/86A406A0A406933B6/HOAF/HOAF_BIDS'            # location of experiment folder
output_dir = '/derivatives/secondlevel/output'     # name of 2nd-level output folder
input_dir_norm = 'derivatives/secondlevel/datasink'# name of norm output folder
working_dir = 'workingdir'# name of working directory
subject_list = [ 'sub006','sub007', 'sub008', 'sub009','sub010']                       # list of subject identifiers
contrast_list = [ 'con_0004']

In [50]:
# One Sample T-Test Design - creates one sample T-Test Design
onesamplettestdes = Node(OneSampleTTestDesign(),
                         name="onesampttestdes")

# EstimateModel - estimate the parameters of the model
level2estimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      name="level2estimate")

# EstimateContrast - estimates simple group contrast
level2conestimate = Node(EstimateContrast(group_contrast=True),
                         name="level2conestimate")
cont1 = ['Group', 'T', ['mean'], [1]]
level2conestimate.inputs.contrasts = [cont1]

In [51]:
# Specify 2nd-Level Analysis Workflow & Connect Nodes
l2analysis = Workflow(name='l2analysis')
l2analysis.base_dir = opj(experiment_dir, working_dir)

# Connect up the 2nd-level analysis components
l2analysis.connect([(onesamplettestdes, level2estimate, [('spm_mat_file',
                                                          'spm_mat_file')] ),
                    (level2estimate, level2conestimate, [('spm_mat_file',
                                                          'spm_mat_file'),
                                                         ('beta_images',
                                                          'beta_images'),
                                                         ('residual_image',
                                                          'residual_image')]),
                    ])


In [52]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['contrast_id']),
                  name="infosource")
infosource.iterables = [('contrast_id', contrast_list)]

# SelectFiles - to grab the data (alternative to DataGrabber)
con_file = opj(input_dir_norm, 'norm_ants', 'sub-*_fwhm4', '{contrast_id}_trans.nii')
templates = {'cons': con_file}

selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_contrast_id_', '')]
datasink.inputs.substitutions = substitutions

In [53]:
# Gunzip - unzip the mask image
#gunzip_con = Node(Gunzip(in_file=mask), name="gunzip_con")

#gunzip_con = MapNode(Gunzip(), name="gunzip_con",
#                     iterfield=['in_file'])

In [54]:
# Connect SelectFiles and DataSink to the workflow
l2analysis.connect([(infosource, selectfiles, [('contrast_id',
                                                'contrast_id')]),
                    (selectfiles, onesamplettestdes, [('cons', 'in_files')]),
                    (level2conestimate, datasink, [('spm_mat_file',
                                                    'contrasts.@spm_mat'),
                                                   ('spmT_images',
                                                    'contrasts.@T'),
                                                   ('con_images',
                                                    'contrasts.@con')]),
                    ])

In [55]:
l2analysis.write_graph(graph2use='colored')
l2analysis.run('MultiProc', plugin_args={'n_procs': 8})

220307-09:42:21,931 nipype.workflow INFO:
	 Generated workflow graph: /media/lmn/86A406A0A406933B6/HOAF/HOAF_BIDS/workingdir/l2analysis/graph.png (graph2use=colored, simple_form=True).
220307-09:42:21,938 nipype.workflow INFO:
	 Workflow l2analysis settings: ['check', 'execution', 'logging', 'monitoring']
220307-09:42:21,953 nipype.workflow INFO:
	 Running in parallel.
220307-09:42:21,956 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.21/28.21, Free processors: 8/8.
220307-09:42:22,27 nipype.workflow INFO:
	 [Node] Setting-up "l2analysis.selectfiles" in "/media/lmn/86A406A0A406933B6/HOAF/HOAF_BIDS/workingdir/l2analysis/_contrast_id_con_0004/selectfiles".
220307-09:42:22,35 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
220307-09:42:22,57 nipype.workflow INFO:
	 [Node] Finished "l2analysis.selectfiles".
220307-09:42:23,958 nipype.workflow INFO:
	 [Job 0] Completed (l2analysis.selectfiles).
220307-09:

/home/lmn/install/miniconda3/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
/home/lmn/install/miniconda3/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


220307-09:42:26,5 nipype.workflow INFO:
	 [Node] Setting-up "l2analysis.level2estimate" in "/media/lmn/86A406A0A406933B6/HOAF/HOAF_BIDS/workingdir/l2analysis/_contrast_id_con_0004/level2estimate".
220307-09:42:26,17 nipype.workflow INFO:
	 [Node] Running "level2estimate" ("nipype.interfaces.spm.model.EstimateModel")
220307-09:42:27,961 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.01/28.21, Free processors: 7/8.
                     Currently running:
                       * l2analysis.level2estimate
220307-09:42:56,872 nipype.workflow WARNING:
	 [Node] Error on "l2analysis.level2estimate" (/media/lmn/86A406A0A406933B6/HOAF/HOAF_BIDS/workingdir/l2analysis/_contrast_id_con_0004/level2estimate)
220307-09:42:57,992 nipype.workflow ERROR:
	 Node level2estimate.a0 failed to run on host lmn-pc.
220307-09:42:57,993 nipype.workflow ERROR:
	 Saving crash info to /home/lmn/crash-20220307-094257-lmn-level2estimate.a0-48cf3f56-e3e9-4cdf-a1c4-a9d4ffa2a

220307-09:42:58,6 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 28.21/28.21, Free processors: 8/8.
220307-09:42:59,991 nipype.workflow INFO:
	 ***********************************
220307-09:42:59,992 nipype.workflow ERROR:
	 could not run node: l2analysis.level2estimate.a0
220307-09:42:59,993 nipype.workflow INFO:
	 crashfile: /home/lmn/crash-20220307-094257-lmn-level2estimate.a0-48cf3f56-e3e9-4cdf-a1c4-a9d4ffa2a37a.pklz
220307-09:42:59,993 nipype.workflow INFO:
	 ***********************************


RuntimeError: Workflow did not execute cleanly. Check log for details